In [16]:
import json
from datetime import datetime
import pandas as pd
from openai_query import consultar_chatgpt
import duckdb as ddbb

In [17]:
with open('resp.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

    final = []

In [18]:
for item in data:
    date = datetime.fromtimestamp(item['timestamp']).date()
    time = datetime.fromtimestamp(item['timestamp']).time()
    id = item['id']
    try:
        body = item['body']
    except:
        body = "Not available"
    try:
        text = item['_data']['Message']['extendedTextMessage']['text']
    except:
        text = "Not available"
    try:
        title = item['_data']['Message']['extendedTextMessage']['title']
    except:
        title = "Not available"
    try:
        description = item['_data']['Message']['extendedTextMessage']['description']
    except:
        description = "Not available"
    try:
        caption = item['_data']['Message']['imageMessage']['caption']
    except:
        caption = "Not available"

    current_intem = {
        'date': str(date),
        'time': str(time),
        'id': id,
        'body': body,
        'text': text,
        'title': title,
        'description': description,
        'caption': caption
    }

    prompt = f"""
    Com base nos dados a seguir, classifique o conteúdo em uma das categorias:

    Eletrodomésticos
    Móveis e Decoração
    Utilidades Domésticas
    Construção e Ferramentas
    Celulares e Smartphones
    Informática
    TV, Áudio e Vídeo
    Games e Consoles
    Moda e Vestuário
    Beleza e Perfumaria
    Relógios e Joias
    Bebês e Maternidade
    Saúde e Cuidados Pessoais
    Pet Shop
    Esporte e Fitness
    Brinquedos e Hobbies
    Viagem e Camping
    Alimentos e Bebidas
    Limpeza e Higiene
    Automotivo

    Analise o seguinte texto: "{current_intem}"

    Retorne apenas a categoria correspondente.

    """

    resultado = consultar_chatgpt(prompt)

    current_intem = {
        'date': str(date),
        'time': str(time),
        'id': id,
        'body': body,
        'text': text,
        'title': title,
        'description': description,
        'caption': caption,
        'categoria': resultado
    }

    final.append(current_intem)


In [19]:
df = pd.DataFrame(final)
df.to_csv('data.csv', index=False)
df

,date,time,id,body,text,title,description,caption,categoria
0,2025-11-03,16:30:40,false_120363257853840248@g.us_3EB001D6A5A33338...,"\n\n🛍️ PAYOT Sabonete Liquido Detox, Vitamina...",Not available,Not available,Not available,"\n\n🛍️ PAYOT Sabonete Liquido Detox, Vitamina...",Beleza e Perfumaria
1,2025-11-03,16:29:11,false_120363257853840248@g.us_3EB08E5AD4A87D8D...,\n\n🛍️ Frigobar 93L INOX 127V Midea\n\n *De:*...,Not available,Not available,Not available,\n\n🛍️ Frigobar 93L INOX 127V Midea\n\n *De:*...,Eletrodomésticos
2,2025-11-03,16:28:11,false_120363257853840248@g.us_3EB07C1B6CFF5F97...,\n\n🛍️ Desodorante em Gel Antitranspirante Se...,Not available,Not available,Not available,\n\n🛍️ Desodorante em Gel Antitranspirante Se...,Beleza e Perfumaria
3,2025-11-03,16:27:00,false_120363257853840248@g.us_3EB0182655334D1E...,\n\n🛍️ Ventisol Ventilador de Coluna Oscilant...,Not available,Not available,Not available,\n\n🛍️ Ventisol Ventilador de Coluna Oscilant...,Eletrodomésticos
4,2025-11-03,16:25:43,false_120363257853840248@g.us_3EB0D30ECACEDF4F...,\n\n🛍️ Apple iPhone 16 Pro (128 GB) – Titânio...,Not available,Not available,Not available,\n\n🛍️ Apple iPhone 16 Pro (128 GB) – Titânio...,Celulares e Smartphones
5,2025-11-03,16:25:29,false_120363257853840248@g.us_3EB0F42FCBA61412...,"\n\n🛍️ Euro Home, ALU8709-VO, Conjunto de pan...",Not available,Not available,Not available,"\n\n🛍️ Euro Home, ALU8709-VO, Conjunto de pan...",Utilidades Domésticas
6,2025-11-03,16:23:15,false_120363257853840248@g.us_3EB0F13B6653062C...,\n\n🛍️ Kärcher Aspirador de Pó Vertical 2 Em ...,Not available,Not available,Not available,\n\n🛍️ Kärcher Aspirador de Pó Vertical 2 Em ...,Eletrodomésticos
7,2025-11-03,16:21:19,false_120363257853840248@g.us_3EB08DD3DA337975...,\n\n🛍️ +Mu Mulkshake Bebida Proteica Láctea UH...,Not available,Not available,Not available,\n\n🛍️ +Mu Mulkshake Bebida Proteica Láctea UH...,Alimentos e Bebidas
8,2025-11-03,16:20:31,false_120363257853840248@g.us_3EB050B651960E4C...,\n\n🛍️ Euro Home - Frigideira 24 cm 2 l de Al...,Not available,Not available,Not available,\n\n🛍️ Euro Home - Frigideira 24 cm 2 l de Al...,Eletrodomésticos
9,2025-11-03,16:16:57,false_120363257853840248@g.us_3EB0ADD2A04E051A...,\n\n🛍️ Smartphone Motorola Edge 50 Ultra 5G -...,Not available,Not available,Not available,\n\n🛍️ Smartphone Motorola Edge 50 Ultra 5G -...,Celulares e Smartphones


In [20]:
base = ddbb.read_csv('data.csv')
base

┌────────────┬──────────┬────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬───────────────┬───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────┐
│    date    │   time   │                                             id                                 

In [21]:
final = ddbb.sql("""select distinct id, date, time,
         case
         when lower(body) like '%mercadolivre%' or lower(text) like '%mercadolivre%' or lower(title) like '%mercadolivre%' or lower(description) like '%mercadolivre%' or lower(caption) like '%mercadolivre%' then 'MERCADO LIVRE'
         when lower(body) like '%magalu%' or lower(text) like '%magalu%' or lower(title) like '%magalu%' or lower(description) like '%magalu%' or lower(caption) like '%magalu%' then 'MAGAZINE LUIZA'
         when lower(body) like '%shopee%' or lower(text) like '%shopee%' or lower(title) like '%shopee%' or lower(description) like '%shopee%' or lower(caption) like '%shopee%' then 'SHOPEE'
         when lower(body) like '%amzn%' or lower(text) like '%amzn%' or lower(title) like '%amzn%' or lower(description) like '%amzn%' or lower(caption) like '%amzn%' then 'AMAZON'
         when lower(body) like '%amazon%' or lower(text) like '%amazon%' or lower(title) like '%amazon%' or lower(description) like '%amazon%' or lower(caption) like '%amazon%' then 'AMAZON'
         else 'OUTROS' end as ORIGEM,
         case
         when lower(body) like '%cupom%' or lower(text) like '%cupom%' or lower(title) like '%cupom%' or lower(description) like '%cupom%' or lower(caption) like '%cupom%' then True
         else False end as CUPOM,
         case
         when id like '%120363257853840248@g.us%' then 'ACHADINHO 🛍️ #16'
         when id like '%120363362432979590@g.us%' then 'Promos Casa & Beleza #05'
         else 'OUTROS' end as GRUPO,
         categoria
          from base""")

In [22]:
final.write_csv('data_final.csv', sep=',')